In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json

# Load the datasets
item_embedding = pd.read_csv('datasets/kaggle/item_embeddings.csv')
course_list = pd.read_csv('datasets/kaggle/course_list.csv')
rating_df = pd.read_csv('datasets/kaggle/rating_df.csv')

# Ensure user IDs are strings
rating_df['user'] = rating_df['user'].astype(str)

# Create a mapping from course code to a unique integer
course_id_map = {code: idx for idx, code in enumerate(course_list['Course List Code'])}
course_name_map = dict(zip(course_list['Course List Code'], course_list['Course Name']))

# Create a mapping from user to a unique integer
user_id_map = {str(uid): idx for idx, uid in enumerate(rating_df['user'].unique())}

# Apply the mappings to the rating data
rating_df['user_id_mapped'] = rating_df['user'].map(user_id_map)
rating_df['course_id_mapped'] = rating_df['item'].map(course_id_map)

# Drop rows with NaN values in the mapped columns (if mapping fails)
rating_df.dropna(subset=['user_id_mapped', 'course_id_mapped'], inplace=True)

# Ensure 'rating' column is binary (0 or 1)
rating_df['rating'] = rating_df['rating'].apply(lambda x: 1 if x > 0 else 0)

# Only keep relevant columns
rating_df = rating_df[['user_id_mapped', 'course_id_mapped', 'rating']]

# Convert mapped IDs to integers
rating_df['user_id_mapped'] = rating_df['user_id_mapped'].astype(int)
rating_df['course_id_mapped'] = rating_df['course_id_mapped'].astype(int)

# Split the data into train, validation, and test sets
train, temp = train_test_split(rating_df, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

# Save the train.txt file
train_file = 'datasets/kaggle/train.txt'
train.to_csv(train_file, sep=' ', header=False, index=False)

# Save the val.txt file
val_file = 'datasets/kaggle/val.txt'
val.to_csv(val_file, sep=' ', header=False, index=False)

# Save the test.txt file
test_file = 'datasets/kaggle/test.txt'
test.to_csv(test_file, sep=' ', header=False, index=False)

# Save the text.txt file (course id and name mapping)
text_file = 'datasets/kaggle/text.txt'
with open(text_file, 'w') as f:
    for code, name in course_name_map.items():
        idx = course_id_map[code]
        f.write(f"{idx} {name}\n")

print("Preprocessing complete. Files saved as train.txt, val.txt, test.txt, and text.txt.")

# Save the mappings for future reference
with open('datasets/kaggle/user_id_map.json', 'w') as f:
    json.dump({str(k): int(v) for k, v in user_id_map.items()}, f)

with open('datasets/kaggle/course_id_map.json', 'w') as f:
    json.dump({str(k): int(v) for k, v in course_id_map.items()}, f)

print("User and course ID mappings saved as JSON files.")

Preprocessing complete. Files saved as train.txt, val.txt, test.txt, and text.txt.
User and course ID mappings saved as JSON files.


In [7]:
import pandas as pd
import numpy as np

# Load the item embedding CSV file
item_embedding_df = pd.read_csv('datasets/kaggle/item_embeddings.csv')

# Extract the embeddings (assuming the first column is 'Course List Code')
course_embeddings = item_embedding_df.iloc[:, 1:].values  # Extracting the embeddings as a NumPy array

# Save the embeddings as a .npy file
np.save('datasets/kaggle/course_embeddings.npy', course_embeddings)

print("course_embeddings.npy file created successfully!")

# Optionally, you can also save the course codes separately
course_codes = item_embedding_df.iloc[:, 0].values
np.save('datasets/kaggle/course_codes.npy', course_codes)

print("course_codes.npy file created successfully!")

course_embeddings.npy file created successfully!
course_codes.npy file created successfully!


In [1]:
import numpy as np
import pandas as pd

# อ่านไฟล์ CSV
item_embeddings_df = pd.read_csv("datasets/kaggle/item_embeddings.csv")

# ลบคอลัมน์ 'Course Name' เพื่อดึงเฉพาะข้อมูล embedding
precomputed_embeddings = item_embeddings_df.drop(columns=['Course Name']).values

# บันทึก embeddings เป็นไฟล์ .npy
np.save("datasets/kaggle/precomputed_embeddings.npy", precomputed_embeddings)

print("Precomputed embeddings saved to 'datasets/kaggle/precomputed_embeddings.npy'")


Precomputed embeddings saved to 'datasets/kaggle/precomputed_embeddings.npy'
